

We create an instance of a SimpleSkyjoEnv environment, call reset() to initialize the game and list the available agents (players):


In [25]:
import numpy as np

from rlskyjo.environment import simple_skyjo_env_v2
from importlib import reload
reload(simple_skyjo_env_v2)

<module 'rlskyjo.environment.simple_skyjo_env_v2' from '/home/michi/skybo_rl/rlskyjo/environment/simple_skyjo_env_v2.py'>

In [2]:
skyjo_env_cfg = {"num_players": 3}
env_pettingzoo = simple_skyjo_env_v2.env(**skyjo_env_cfg)
env_pettingzoo.reset()

env_pettingzoo.agents, env_pettingzoo.agent_selection

def sample_place():
    return np.random.randint(0,23)
        
def sample_draw():
    return np.random.randint(24,25)


In [3]:
print(env_pettingzoo.agent_selection, "draw")
env_pettingzoo.step(sample_draw())
print(env_pettingzoo.agent_selection, "place")
env_pettingzoo.step(
    sample_place()
)

player_2 draw
player_2 place


In [4]:
def random_admissible_policy(observation, action_mask):
    """picks randomly an admissible action from the action mask"""
    return np.random.choice(
        np.arange(len(action_mask)),
        p= action_mask/np.sum(action_mask)
    )

assert 1 not in [random_admissible_policy(None,[1,0,1]) for _ in range(1000)] 

In [5]:
i_episode = 1  
while i_episode <= 1:
    i_episode  += 1
    env_pettingzoo.reset()
    for agent in env_pettingzoo.agent_iter(max_iter=600):        
        # get observation (state) for current agent:
        obs, reward, done, info = env_pettingzoo.last()
        
        print("training fct:", obs, reward, done, info)
        # perform q-learning with update_Q_value()
        # your code here
        
        env_pettingzoo.render()
        
        # store current state            
        if not done: 
            # choose action using epsilon_greedy_policy()
            # your code here    
            observation = obs["observations"]
            action_mask = obs["action_mask"]
            action = random_admissible_policy(observation, action_mask)
        
            print(f"sampled action {agent}: {action}")
            env_pettingzoo.step(action)
        else: 
            # agent is done
            env_pettingzoo.step(None)
            print('done', reward)
            break
    
    
else:
    print(env_pettingzoo._cumulative_rewards)

training fct: {'observations': array([15, 15, 15,  8,  3, 15, 15, 15, 15, 15, 15, 15,  3, 10,  0,  0,  2,
        0,  0,  1,  0,  1,  0,  0,  1,  1,  0,  1,  0, 11, 15], dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 1 
holding card player 1: empty 
discard pile top: 11 
======= Player 0 ========== 
 [x	x	x	5]
 [9	x	x	x]
 [x	x	x	x]  
======= Player 1 ========== 
 [x	x	x	8]
 [x	x	x	x]
 [x	0	x	x]  
======= Player 2 ========== 
 [x	x	x	x]
 [x	x	x	0]
 [x	x	3	x]  

sampled action player_1: 24
training fct: {'observations': array([15, 15, 15,  8,  3, 15, 15, 15, 15, 15, 15, 15,  3, 10,  0,  0,  2,
        0,  0,  1,  0,  1,  0,  0,  1,  1,  0,  1,  0, 11, -2], dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0], dtype=int8)} 0 False {}
======= ren

more envs test with rllib

In [6]:
from ray import init
init(num_cpus=os.cpu_count())

{'node_ip_address': '172.17.87.73',
 'raylet_ip_address': '172.17.87.73',
 'redis_address': '172.17.87.73:64228',
 'object_store_address': '/tmp/ray/session_2022-01-28_00-10-44_149462_2156/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-28_00-10-44_149462_2156/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-28_00-10-44_149462_2156',
 'metrics_export_port': 43368,
 'node_id': '8ff9d6fb5900ae0f0508f4e7852164560a041347c64fd4442910b6f3'}

In [7]:
from ray.tune.registry import register_env
from rlskyjo.environment import simple_skyjo_env_v2
from ray.rllib.env import PettingZooEnv
import ray.rllib.agents.dqn as dqn
from ray.rllib.agents import ppo
from copy import deepcopy
from ray.rllib.agents.registry import get_agent_class
from ray.rllib.models import ModelCatalog
from rlskyjo.models.action_mask_model import TorchMaskedActions, TorchActionMaskModel
from ray.rllib.models import ModelCatalog
from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.tune.logger import pretty_print
import os


In [26]:

env_name  = "pettingzoo_skyjo"
def env_creator():
        env = simple_skyjo_env_v2.env(**skyjo_env_cfg)
        return env

register_env(env_name,
                lambda config: PettingZooEnv(env_creator()))
ModelCatalog.register_custom_model(
        "pa_model2", TorchActionMaskModel
    )
env = PettingZooEnv(env_creator())

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [9]:
env.env.observe(env.env.agent_selection)

{'observations': array([ 3, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 11,  2, 10,  0,  1,  1,
         1,  1,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1,  6, 15], dtype=int8),
 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1], dtype=int8)}

with multiagent

In [10]:
custom_config={
    "env":env_name,
    "model": {
        "custom_model": "pa_model2",
    },
    "framework": "torch",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "num_workers": os.cpu_count(),
    "multiagent":{
            "policies": {
                name: (None, env.observation_space, env.action_space, {}) for name in env.agents
            },
            "policy_mapping_fn": lambda agent_id: agent_id
    }
}


ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(custom_config)

trainer = ppo.PPOTrainer(config=ppo_config)

2022-01-28 00:10:48,936	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=16 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 250.
2022-01-28 00:10:48,937	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-01-28 00:10:48,938	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=7794) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
(RolloutWorker pid=7794)   warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
(RolloutWorker pid=7794) /home/m

In [11]:
# run manual training loop and print results after each iteration
max_steps = 1e7
max_iters = 500
for iters in range(max_iters):
    result = trainer.train()
    if iters % 2 ==0:
        print(pretty_print(result))
    # stop training if the target train steps or reward are reached
    if result["timesteps_total"] >= max_steps:
        print(f"training done, because max_steps {max_steps} {result['timesteps_total']} reached")
        break
else:
    print(f"training done, because max_iters {max_iters} reached")
# manual test loop
print("Finished training. Running manual test/inference loop.")
# prepare environment with max 10 steps


(RolloutWorker pid=7785) 2022-01-28 00:11:05,530	WARNING deprecation.py:45 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!
(RolloutWorker pid=7785) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=7785)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
(RolloutWorker pid=7785) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=7785)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


agent_timesteps_total: 3968
custom_metrics: {}
date: 2022-01-28_00-11-13
done: false
episode_len_mean: 100.875
episode_media: {}
episode_reward_max: 3.0000000000000284
episode_reward_mean: 3.0000000000000027
episode_reward_min: 2.9999999999999716
episodes_this_iter: 32
episodes_total: 32
experiment_id: 1cf34fbceba648faabb88c0799df1a9f
hostname: lenovo-michi
info:
  learner:
    player_0:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.19999999999999998
        cur_lr: 5.000000000000001e-05
        entropy: 1.7334793674945832
        entropy_coeff: 0.0
        kl: 0.006546088004887527
        policy_loss: -0.06398468928101163
        total_loss: 124.69429440816243
        vf_explained_var: 0.004620983203252157
        vf_loss: 124.75696977615357
      model: {}
    player_1:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.19999999999999998
        cur_lr: 5.000000000000001e-05
     

In [38]:
env = PettingZooEnv(env_creator())
obs = env.reset()
done = {'__all__': False}
# run one iteration until done
for i in range(1):
    if done["__all__"]:
        print("game done")
        break
    # get agent from current observation
    agent = list(obs.keys())[0]

    # format observation dict
    obs = obs[agent]
    env.render()
    
    # get deterministic action
    # trainer.compute_single_action(obs, policy_id=agent)
    policy = trainer.get_policy(policy_id=agent)
    action_exploration_policy, _, action_info = policy.compute_single_action(obs)
    logits = action_info['action_dist_inputs']
    action = logits.argmax()
    # 
    print("agent ", agent, " action ", env.env.env.table.render_action_explainer(action                                                           ))
    next_obs, reward, done, obs = env.step({agent: action})
    # observations contain original observations and the action mask
    # print(f"Obs: {obs}, Action: {action}, done: {done}")
    
env.render()
    

======= render board: ===== 
======= stats ============ 
next turn: draw by Player 0 
holding card player 0: empty 
discard pile top: 11 
======= Player 0 ========== 
 [x	x	x	x]
 [x	x	x	x]
 [x	12	0	x]  
======= Player 1 ========== 
 [x	x	x	x]
 [x	x	x	x]
 [x	10	1	x]  
======= Player 2 ========== 
 [5	x	x	x]
 [x	7	x	x]
 [x	x	x	x]  

agent  player_0  action  draw from discard pile
======= render board: ===== 
======= stats ============ 
next turn: place by Player 0 
holding card player 0: 11 
discard pile top: empty 
======= Player 0 ========== 
 [x	x	x	x]
 [x	x	x	x]
 [x	12	0	x]  
======= Player 1 ========== 
 [x	x	x	x]
 [x	x	x	x]
 [x	10	1	x]  
======= Player 2 ========== 
 [5	x	x	x]
 [x	7	x	x]
 [x	x	x	x]  



/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [13]:
obs = {'observation': np.array([15, 15,  7,  4, 15, -2, 15,  6, 12, 15, 15,  4,  9,  6,  3,  0,  1,
        0,  1,  0,  2,  1,  1,  2,  0,  1,  1,  0,  1, -2,  4], dtype=np.int8), 'action_mask': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=np.int8)}
action = trainer.compute_single_action(obs, policy_id=agent)

ValueError: ('Observation ({} dtype={}) outside given space ({})!', {'observation': array([15, 15,  7,  4, 15, -2, 15,  6, 12, 15, 15,  4,  9,  6,  3,  0,  1,
        0,  1,  0,  2,  1,  1,  2,  0,  1,  1,  0,  1, -2,  4], dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int8)}, None, Dict(action_mask:Box([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], (26,), int8), observations:Box([-24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24
 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24], [127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127
 127 127 127 127 127 127 127 127 127 127 127 127 127], (31,), int8)))

E0128 01:17:45.539222787   21306 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643329065.539180366","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643329065.539174615","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


Try creating with rllib